# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
#%%capture
#%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::029532071818:role/service-role/AmazonSageMaker-ExecutionRole-20231210T215923


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://minhnbbucket/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
#git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
#cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
#cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [6]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
#image_name = 'tf2-object-detection'
#!sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

029532071818.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20231210132738


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [8]:
%%bash
mkdir /tmp/efficient_net_checkpoint
mkdir source_dir/efficient_net_checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/efficient_net_checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

mkdir: cannot create directory ‘source_dir/efficient_net_checkpoint’: File exists
--2023-12-11 05:18:01--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.115.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘/tmp/efficientdet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 13.7M 4s
    50K .......... .......... .......... .......... ..........  0% 28.6M 3s
   100K .......... .......... .......... .......... ..........  0% 25.0M 2s
   150K .......... .......... .......... .......... ..........  0% 61.1M 2s
   200K .......... .......... .......... .......... ..........  0% 28.1M 2s
   250K .......... .......... .......... .......... ..........  

  5050K .......... .......... .......... .......... .......... 10% 58.9M 1s
  5100K .......... .......... .......... .......... .......... 10%  127M 1s
  5150K .......... .......... .......... .......... .......... 10% 65.1M 1s
  5200K .......... .......... .......... .......... .......... 10% 53.0M 1s
  5250K .......... .......... .......... .......... .......... 10% 45.8M 1s
  5300K .......... .......... .......... .......... .......... 10%  519M 1s
  5350K .......... .......... .......... .......... .......... 10% 63.0M 1s
  5400K .......... .......... .......... .......... .......... 10% 44.8M 1s
  5450K .......... .......... .......... .......... .......... 10% 66.2M 1s
  5500K .......... .......... .......... .......... .......... 10% 56.9M 1s
  5550K .......... .......... .......... .......... .......... 11%  132M 1s
  5600K .......... .......... .......... .......... .......... 11% 56.6M 1s
  5650K .......... .......... .......... .......... .......... 11% 53.2M 1s
  5700K ....

 10450K .......... .......... .......... .......... .......... 20%  233M 1s
 10500K .......... .......... .......... .......... .......... 20% 71.1M 1s
 10550K .......... .......... .......... .......... .......... 20% 75.2M 1s
 10600K .......... .......... .......... .......... .......... 21% 69.9M 1s
 10650K .......... .......... .......... .......... .......... 21% 90.2M 1s
 10700K .......... .......... .......... .......... .......... 21%  191M 1s
 10750K .......... .......... .......... .......... .......... 21% 87.3M 1s
 10800K .......... .......... .......... .......... .......... 21% 70.1M 1s
 10850K .......... .......... .......... .......... .......... 21% 69.5M 1s
 10900K .......... .......... .......... .......... .......... 21%  101M 1s
 10950K .......... .......... .......... .......... .......... 21%  248M 1s
 11000K .......... .......... .......... .......... .......... 21% 85.0M 1s
 11050K .......... .......... .......... .......... .......... 21% 72.7M 1s
 11100K ....

 15850K .......... .......... .......... .......... .......... 31%  146M 0s
 15900K .......... .......... .......... .......... .......... 31% 81.2M 0s
 15950K .......... .......... .......... .......... .......... 31%  257M 0s
 16000K .......... .......... .......... .......... .......... 31%  153M 0s
 16050K .......... .......... .......... .......... .......... 31%  332M 0s
 16100K .......... .......... .......... .......... .......... 31%  134M 0s
 16150K .......... .......... .......... .......... .......... 32%  101M 0s
 16200K .......... .......... .......... .......... .......... 32%  142M 0s
 16250K .......... .......... .......... .......... .......... 32%  154M 0s
 16300K .......... .......... .......... .......... .......... 32%  283M 0s
 16350K .......... .......... .......... .......... .......... 32%  120M 0s
 16400K .......... .......... .......... .......... .......... 32%  138M 0s
 16450K .......... .......... .......... .......... .......... 32%  155M 0s
 16500K ....

 21250K .......... .......... .......... .......... .......... 42%  251M 0s
 21300K .......... .......... .......... .......... .......... 42%  344M 0s
 21350K .......... .......... .......... .......... .......... 42%  271M 0s
 21400K .......... .......... .......... .......... .......... 42%  297M 0s
 21450K .......... .......... .......... .......... .......... 42%  321M 0s
 21500K .......... .......... .......... .......... .......... 42%  364M 0s
 21550K .......... .......... .......... .......... .......... 42%  358M 0s
 21600K .......... .......... .......... .......... .......... 42%  264M 0s
 21650K .......... .......... .......... .......... .......... 42%  260M 0s
 21700K .......... .......... .......... .......... .......... 42%  257M 0s
 21750K .......... .......... .......... .......... .......... 43%  396M 0s
 21800K .......... .......... .......... .......... .......... 43%  269M 0s
 21850K .......... .......... .......... .......... .......... 43%  333M 0s
 21900K ....

...

 32050K .......... .......... .......... .......... .......... 63%  530M 0s
 32100K .......... .......... .......... .......... .......... 63%  541M 0s
 32150K .......... .......... .......... .......... .......... 63%  523M 0s
 32200K .......... .......... .......... .......... .......... 63%  456M 0s
 32250K .......... .......... .......... .......... .......... 63%  533M 0s
 32300K .......... .......... .......... .......... .......... 63%  443M 0s
 32350K .......... .......... .......... .......... .......... 64%  489M 0s
 32400K .......... .......... .......... .......... .......... 64%  448M 0s
 32450K .......... .......... .......... .......... .......... 64%  515M 0s
 32500K .......... .......... .......... .......... .......... 64%  482M 0s
 32550K .......... .......... .......... .......... .......... 64%  552M 0s
 32600K .......... .......... .......... .......... .......... 64%  422M 0s
 32650K .......... .......... .......... .......... .......... 64%  533M 0s
 32700K ....

 37450K .......... .......... .......... .......... .......... 74%  278M 0s
 37500K .......... .......... .......... .......... .......... 74%  236M 0s
 37550K .......... .......... .......... .......... .......... 74%  191M 0s
 37600K .......... .......... .......... .......... .......... 74%  152M 0s
 37650K .......... .......... .......... .......... .......... 74%  140M 0s
 37700K .......... .......... .......... .......... .......... 74%  144M 0s
 37750K .......... .......... .......... .......... .......... 74%  157M 0s
 37800K .......... .......... .......... .......... .......... 74%  153M 0s
 37850K .......... .......... .......... .......... .......... 74%  271M 0s
 37900K .......... .......... .......... .......... .......... 74%  212M 0s
 37950K .......... .......... .......... .......... .......... 75%  260M 0s
 38000K .......... .......... .......... .......... .......... 75%  498M 0s
 38050K .......... .......... .......... .......... .......... 75%  522M 0s
 38100K ....

 42850K .......... .......... .......... .......... .......... 84%  358M 0s
 42900K .......... .......... .......... .......... .......... 84%  185M 0s
 42950K .......... .......... .......... .......... .......... 84%  230M 0s
 43000K .......... .......... .......... .......... .......... 85%  251M 0s
 43050K .......... .......... .......... .......... .......... 85%  260M 0s
 43100K .......... .......... .......... .......... .......... 85%  328M 0s
 43150K .......... .......... .......... .......... .......... 85%  258M 0s
 43200K .......... .......... .......... .......... .......... 85%  271M 0s
 43250K .......... .......... .......... .......... .......... 85%  288M 0s
 43300K .......... .......... .......... .......... .......... 85%  217M 0s
 43350K .......... .......... .......... .......... .......... 85%  302M 0s
 43400K .......... .......... .......... .......... .......... 85%  249M 0s
 43450K .......... .......... .......... .......... .......... 85%  395M 0s
 43500K ....

 48250K .......... .......... .......... .......... .......... 95%  523M 0s
 48300K .......... .......... .......... .......... .......... 95%  483M 0s
 48350K .......... .......... .......... .......... .......... 95%  501M 0s
 48400K .......... .......... .......... .......... .......... 95%  453M 0s
 48450K .......... .......... .......... .......... .......... 95%  555M 0s
 48500K .......... .......... .......... .......... .......... 95%  493M 0s
 48550K .......... .......... .......... .......... .......... 96%  535M 0s
 48600K .......... .......... .......... .......... .......... 96%  460M 0s
 48650K .......... .......... .......... .......... .......... 96%  510M 0s
 48700K .......... .......... .......... .......... .......... 96%  520M 0s
 48750K .......... .......... .......... .......... .......... 96%  545M 0s
 48800K .......... .......... .......... .......... .......... 96%  415M 0s
 48850K .......... .......... .......... .......... .......... 96%  561M 0s
 48900K ....

efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d1_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [10]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "pipeline_efficient.config",
        "num_train_steps": "1000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='efficient-net-tf2-object-detection'
)

estimator.fit(inputs)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: efficient-net-tf2-object-detection-2023-12-10-17-09-16-327


2023-12-10 17:09:30 Starting - Starting the training job...
2023-12-10 17:09:47 Starting - Preparing the instances for training......
2023-12-10 17:10:57 Downloading - Downloading input data...
2023-12-10 17:11:27 Training - Downloading the training image.........
2023-12-10 17:12:58 Training - Training image download completed. Training in progress.....2023-12-10 17:13:29,283 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-10 17:13:29,321 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-10 17:13:29,360 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-10 17:13:29,373 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current_in

I1210 17:13:36.867623 139663435204416 efficientnet_model.py:143] round_filter input=40 output=40
I1210 17:13:36.867749 139663435204416 efficientnet_model.py:143] round_filter input=80 output=80
I1210 17:13:37.260724 139663435204416 efficientnet_model.py:143] round_filter input=80 output=80
I1210 17:13:37.260850 139663435204416 efficientnet_model.py:143] round_filter input=112 output=112
I1210 17:13:37.644131 139663435204416 efficientnet_model.py:143] round_filter input=112 output=112
I1210 17:13:37.644258 139663435204416 efficientnet_model.py:143] round_filter input=192 output=192
I1210 17:13:38.126403 139663435204416 efficientnet_model.py:143] round_filter input=192 output=192
I1210 17:13:38.126532 139663435204416 efficientnet_model.py:143] round_filter input=320 output=320
I1210 17:13:38.325258 139663435204416 efficientnet_model.py:143] round_filter input=1280 output=1280
I1210 17:13:38.410023 139663435204416 efficientnet_model.py:453] Building model efficientnet with params ModelCon

W1210 17:14:36.089058 139633748866816 utils.py:82] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss` argument?
I1210 17:14:39.798425 139633748866816 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
W1210 17:14:45.714224 139633748866816 utils.py:82] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/b

INFO:tensorflow:Step 900 per-step time 0.662s
I1210 17:25:17.547815 139663435204416 model_lib_v2.py:705] Step 900 per-step time 0.662s
INFO:tensorflow:{'Loss/classification_loss': 0.27920803,
 'Loss/localization_loss': 0.014317824,
 'Loss/regularization_loss': 0.029670754,
 'Loss/total_loss': 0.3231966,
 'learning_rate': 0.02944}
I1210 17:25:17.548057 139663435204416 model_lib_v2.py:708] {'Loss/classification_loss': 0.27920803,
 'Loss/localization_loss': 0.014317824,
 'Loss/regularization_loss': 0.029670754,
 'Loss/total_loss': 0.3231966,
 'learning_rate': 0.02944}
INFO:tensorflow:Step 1000 per-step time 0.662s
I1210 17:26:23.765596 139663435204416 model_lib_v2.py:705] Step 1000 per-step time 0.662s
INFO:tensorflow:{'Loss/classification_loss': 0.31346253,
 'Loss/localization_loss': 0.01774502,
 'Loss/regularization_loss': 0.02970879,
 'Loss/total_loss': 0.36091635,
 'learning_rate': 0.0326}
I1210 17:26:23.765841 139663435204416 model_lib_v2.py:708] {'Loss/classification_loss': 0.313462

/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
I1210 17:26:49.556298 140520511784768 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I1210 17:27:02.506939 140520511784768 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
Instructions for updating:
Use `tf.cast` instead.
W1210 17:27:07.738952 140520511784768 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Finished eval step 0
I1210 17:27:07.832282 140520511784768 model_lib_v2.py:966] Fin

I1210 17:31:55.276384 140038104020800 ssd_efficientnet_bifpn_feature_extractor.py:161] EfficientDet EfficientNet backbone version: efficientnet-b1
I1210 17:31:55.276531 140038104020800 ssd_efficientnet_bifpn_feature_extractor.py:163] EfficientDet BiFPN num filters: 88
I1210 17:31:55.276589 140038104020800 ssd_efficientnet_bifpn_feature_extractor.py:164] EfficientDet BiFPN num iterations: 4
I1210 17:31:55.280093 140038104020800 efficientnet_model.py:143] round_filter input=32 output=32
I1210 17:31:55.311546 140038104020800 efficientnet_model.py:143] round_filter input=32 output=32
I1210 17:31:55.311680 140038104020800 efficientnet_model.py:143] round_filter input=16 output=16
I1210 17:31:55.443293 140038104020800 efficientnet_model.py:143] round_filter input=16 output=16
I1210 17:31:55.443422 140038104020800 efficientnet_model.py:143] round_filter input=24 output=24
I1210 17:31:55.678767 140038104020800 efficientnet_model.py:143] round_filter input=24 output=24
I1210 17:31:55.678900 140

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your write-up goes here.